##### 导入库

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE,SMOTE
from sklearn.utils import shuffle

import box
import json

##### 选择数据

In [44]:
def get_fea(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
    # path = 'E:\\desktop\\data\\app\\fea\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data


n='n.csv'
p={
    "all":'all.csv',
    "cns":'cns.csv',
    "opi":'opi.csv',
    "other":'other.csv'
}

way="other"

##### 处理数据

In [45]:
n_fea = get_fea(n)
p_fea = get_fea(p[way])


fea_dict = {
    "all":
[
  "fft_real_2",
  "alt_stderr_min_50",
  "alt_rvalue_min_50",
  "minindex",
  "fft_angle_3",
  "erbc_num10_focus2",
  "fft_imag_3",
  "lumpiness"
],

    "opi":[
  "fft_real_2",
  "minindex",
  "alt_stderr_min_50",
  "alt_rvalue_min_50",
  "lumpiness",
  "erbc_num10_focus2",
  "fft_imag_3"
],
    "cns":[
  "alt_rvalue_min_50",
  "fft_angle_2",
  "S_down",
  "fft_imag_3",
  "minindex",
  "alt_stderr_min_50",
  "fft_angle_3",
  "swd_coeff_2",
  "erbc_num10_focus2",
  "erbc_num10_focus4",
  "lumpiness"
],
    "other":[
  "fft_imag_3",
  "minindex",
  "alt_rvalue_min_50",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_real_2",
  "erbc_num10_focus4",
  "retindex",
  "alt_stderr_max_5",
  "cv",
  "asoc",
  "minvalue",
  "erbc_num10_focus2"
]
}



fea_list=fea_dict[way]
n_fea=n_fea[fea_list]
p_fea=p_fea[fea_list]

n_fea['target']=[0 for i in range(len(n_fea))]
p_fea['target']=[1 for i in range(len(p_fea))]

np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))


##### 划分数据&分箱

In [46]:
proportion=0.3 #验证集占比
X=np_fea[fea_list]
Y=np_fea[['target']]

#过采样
ov=SMOTE(random_state=0)
X,Y=ov.fit_resample(X,Y)

##### 验证结果

In [47]:
while 1:
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=proportion) #,random_state=0 取消随机
    XY_train=pd.concat([X_train,Y_train],axis=1)
    XY_verify=pd.concat([X_test,Y_test],axis=1)
    n_train=XY_train[XY_train['target']==0]
    p_train=XY_train[XY_train['target']==1]
    n_verify=XY_verify[XY_verify['target']==0]
    p_verify=XY_verify[XY_verify['target']==1]

    #分箱
    config=box.divide(n_train,p_train,fea_list)

    n_jugde=np.sum(box.jugde(n_verify,config))
    p_jugde=np.sum(box.jugde(p_verify,config))

    n_jugdes=np.sum(box.jugde(n_fea,config))
    p_jugdes=np.sum(box.jugde(p_fea,config))

    if n_jugdes/len(n_fea)<0.1: #p_jugdes/len(p_fea)>0.77 and 
        print("测试集")
        print("检出率: ",p_jugde,"/",len(p_verify),"  ",p_jugde/len(p_verify))
        print("误判率: ",n_jugde,"/",len(n_verify),"  ",n_jugde/len(n_verify))
        print("精确率: ",len(n_verify)-n_jugde+p_jugde,"/",len(n_verify)+len(p_verify),"  ",(len(n_verify)-n_jugde+p_jugde)/(len(n_verify)+len(p_verify)))

        print('-'*50)
        print("总数据")
        print("检出率: ",p_jugdes,"/",len(p_fea),"  ",p_jugdes/len(p_fea))
        print("误判率: ",n_jugdes,"/",len(n_fea),"  ",n_jugdes/len(n_fea))
        print("精确率: ",len(n_fea)-n_jugdes+p_jugdes,"/",len(n_fea)+len(p_fea),"  ",(len(n_fea)-n_jugdes+p_jugdes)/(len(n_fea)+len(p_fea)))
        break

测试集
检出率:  125 / 166    0.7530120481927711
误判率:  19 / 147    0.1292517006802721
精确率:  253 / 313    0.8083067092651757
--------------------------------------------------
总数据
检出率:  74 / 90    0.8222222222222222
误判率:  47 / 521    0.09021113243761997
精确率:  548 / 611    0.8968903436988543


##### 保存结果

In [48]:
def default_dump(obj):
    """Convert numpy classes to JSON serializable objects."""
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

config_json=json.dumps(config,default=default_dump)#转化为json格式文件

with open('./config/'+way+'.json','w+') as file:
    file.write(config_json)

print(way,"保存成功")

other 保存成功
